In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns# popular Python visualization librar

In [2]:
import warnings #Warnings are not the same as errors — they don’t stop your program, but they inform you that something may not work in the future or could be problematic.
warnings.filterwarnings("ignore")#The filterwarnings function controls whether warnings are shown or hidden.
#"ignore" is one of several options 
sns.set(style="whitegrid")#Set Seaborn plots to have a clean white background with a grid.

In [5]:
dataset=pd.read_csv(r"D:\dataset\fraudDetection\AIML Dataset.csv")

In [6]:
dataset.head(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [7]:
dataset.shape

(6362620, 11)

In [8]:
dataset.info() #a Pandas DataFrame method that gives you a quick summary of your dataset’s structure.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [9]:
dataset.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [11]:
dataset["isFraud"].value_counts() #gives total count

isFraud
0    6354407
1       8213
Name: count, dtype: int64

In [12]:
dataset["isFlaggedFraud"].value_counts()

isFlaggedFraud
0    6362604
1         16
Name: count, dtype: int64

In [16]:
dataset.isnull().sum() #gives total number of null values as per columns

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [17]:
dataset.isnull().sum().sum() #total number of null value in the entire dataset

np.int64(0)

In [22]:
round((dataset["isFraud"].value_counts()[1]/dataset.shape[0])*100,2)
#dataset.shape[0] → total number of rows in the dataset (total transactions).
#Percentage (in decimal form) of fraud cases in the dataset.

np.float64(0.13)